$\textbf{\large Using the Functions in TrajectoryClassifier module}$

In [18]:
import numpy as np
import math
from stochastic.processes.continuous import WienerProcess
from stochastic.processes.continuous import FractionalBrownianMotion
from stochastic.processes.continuous import BrownianMotion
from stochastic.processes.diffusion import OrnsteinUhlenbeckProcess
from scipy.optimize import minimize_scalar


1. $\textbf{Computing the test statistcs:}$

In [19]:
def maximal_excursion_test_stats(traj, d):

    """ 
    Input parameter:
    -------
    traj : single trajectory from a stochatsic process
    d : dimension of the stochastic process

    Output result:
    -------
    This function will return the maximal excursion test statistics of the input trajectory

    """

    traj = traj.reshape(-1,d)
    stats = (np.sqrt(d) * np.max(np.linalg.norm(traj[1:] - traj[0], axis = 1))) / (np.sqrt(np.sum(np.square(np.linalg.norm(np.diff(traj, axis = 0), axis = 1)))))
    return stats


In [4]:
# Example: Compute the test stats of a BM trajectory of dim 1 with size 10

std_bm = WienerProcess(t = 10)
std_bm_traj = std_bm.sample(10)
maximal_excursion_test_stats(std_bm_traj, d = 1)

0.6371353765566944

In [6]:
# Example: Compute the test stats of a fBM trajectory of dim 2 with size 20 and hurst index = 0.85

super_fbm = FractionalBrownianMotion(hurst = 0.85, t = 20)
super_fbm_traj = np.dstack([[super_fbm.sample(20)], [super_fbm.sample(20)]])
maximal_excursion_test_stats(super_fbm_traj, d = 2)

1.9295355236470642

2. $\textbf{Computing the MSD estimates of 1D trajectory:}$

In [7]:
def MSD_estimate1d(traj, traj_length) : 
    """
    Input parameter:
    -------
    traj: single trajectory from a 1d stochastic process
    traj_length: the length of the trajectory

    Output result:
    -------
    This function will return the MSD estimates of the input trajectory

    """
    output_array = np.zeros(traj_length + 1)
    for lag in range(1,traj_length + 1):
        squared_distances = [(traj[i] - traj[i-lag]) ** 2 for i in range(lag,traj_length + 1)]
        output_array[lag] = np.sum(squared_distances) / (traj_length - lag + 1)
    return output_array

In [8]:
# Example: Compute the MSD estimates of a BM trajectory of dim 1 with size 10

std_bm = WienerProcess(t = 10)
std_bm_traj = std_bm.sample(10)
MSD_estimate1d(std_bm_traj, 10)

array([ 0.        ,  1.03198095,  2.12582867,  2.95922414,  3.49717114,
        4.10197911,  5.94362021,  8.4225104 , 11.28874932, 12.82304485,
       22.04663617])

In [9]:
# Example: Compute the MSD estimates of a OU trajectory of dim 1 with size 20 and speed = 0.53

ou = OrnsteinUhlenbeckProcess(speed = 0.53, t = 20)
ou_traj = ou.sample(20)
MSD_estimate1d(ou_traj, 20)

array([0.        , 1.06446277, 1.25500166, 1.35529989, 2.26169258,
       2.89928538, 3.28345746, 3.64916118, 4.07243569, 4.38676159,
       4.00919071, 4.37726035, 4.33989738, 3.95183286, 3.56763462,
       3.91483852, 3.61299789, 1.28434088, 3.11719906, 3.48364676,
       0.20647917])

3. $\textbf{Computing the MSD estimates of 2D trajectory:}$

In [10]:
def MSD_estimate2d(traj,traj_length): 
    """
    Input parameter:
    -------
    traj: single trajectory from a 2d stochastic process
    traj_length: the length of the trajectory

    Output result:
    -------
    This function will return the MSD estimates of the input trajectory

    """
    traj = traj.reshape(-1,2)
    output_array = np.zeros(traj_length + 1)
    for lag in range(1,traj_length + 1):
        squared_distances = [((traj[i,0] - traj[i-lag,0]) ** 2 + (traj[i,1] - traj[i-lag,1]) ** 2) for i in range(lag,traj_length + 1)]
        output_array[lag] = np.sum(squared_distances) / (traj_length - lag + 1)
    return output_array

In [11]:
# Example: Compute the MSD estimates of a fBM trajectory of dim 2 with size 20 and hurst index = 0.85

super_fbm = FractionalBrownianMotion(hurst = 0.85, t = 20)
super_fbm_traj = np.dstack([[super_fbm.sample(20)], [super_fbm.sample(20)]])
MSD_estimate2d(super_fbm_traj, 20)

array([  0.        ,   1.83430122,   4.83098124,   8.29904154,
        11.3172328 ,  15.5245898 ,  20.55491256,  28.01893184,
        33.77238212,  38.11576635,  41.39049796,  45.41516987,
        51.29919095,  56.37728934,  61.77706239,  69.28987866,
        78.23264183,  89.75373768, 110.47802527, 134.12762107,
       168.00520311])

In [12]:
# Example: Compute the MSD estimates of a BM trajectory of dim 2 with size 10 

std_bm = WienerProcess(t = 10)
std_bm_traj = np.dstack([[std_bm.sample(10)], [std_bm.sample(10)]])
MSD_estimate2d(std_bm_traj, 10)

array([ 0.        ,  2.12886904,  6.13124706, 10.03436857, 12.17277468,
       15.18057976, 17.02756497, 18.41373105, 13.59362331, 10.91085486,
       10.62622122])

4. $\textbf{Asymptotic CDF of test stats in}$ $ d = 1$ $\textbf{under the null}:$

In [13]:
def F_asym1d(x, r = 20000):
    """
    Input parameter:
    -------
    x: test statistics
    r: number of truncated terms in the analytic expression of the asymptotic CDF. By default, r = 20000

    Output result: 
    -------
    The (approximate) value of asymptotic CDF evaluated at x. 

    Remarks:
    -------
    This function only computes the CDF of test stats when dimension = 1. For dimension = 2 and 3, the analytic 
    expression of CDF will be different. 

    """
    a = np.arange(r)
    b = a[::2]
    s = 0
    for i in range(len(b)):
        s = s + ((-1)**(b[i]/2))*(4/((b[i]+1)*(np.pi)))*(math.exp((-1/8)*(((np.pi)*(b[i]+1))/x)**2))
    return s

In [14]:
# Example: Evaluate the CDF at x = 1.56784

F_asym1d(1.56784)

0.7661681598178136

5. $\textbf{Table of the empirical distributions}$ ($d = 2):$ 

In [16]:
def F_hat2d(n, x, func, num_of_MC = 10000):

    """
    Input parameters:
    -------
    n: This is an array of consecutive integers, starts from 2, ends with 1+len(n).
    x: Usually x = np.arange(0,5.1,0.1).
    func: This is a function parameter. It should compute the maximal excursion test stats of the input trajectories.
    num_of_MC: Number of Monte Carlo Simulations. By default, num_of_MC = 10000.

    Output result:
    -------
    This function will return a table, with ith row being the empirical CDF of T_{i+1} evaluated at x.
    Here, T_{i+1} is the maximal excursion test stats of trajectories with size i+1

    """
    table = np.zeros([len(n), len(x)])
    for i in range(len(n)):
        std_bm = WienerProcess(t = int(i+2))
        std_bm_traj = np.dstack([[std_bm.sample(int(i+2)) for k in range(num_of_MC)], [std_bm.sample(int(i+2)) for k in range(num_of_MC)]])
        stats = func(std_bm_traj)
        for j in range(len(x)):
            table[i, j] = np.sum(np.where(stats <= x[j], 1, 0)) / num_of_MC
    return table


def test_stats(traj): 
    result = np.zeros(len(traj))
    for i in range(len(traj)):
        result[i] = (np.sqrt(2) * np.max(np.linalg.norm(traj[i,:,:][1:] - traj[i,:,:][0], axis = 1))) / (np.sqrt(np.sum(np.square(np.linalg.norm(np.diff(traj[i,:,:], axis = 0), axis = 1)))))
    return result


In [ ]:
# Example 

n = np.arange(2, 11)
x = np.arange(0,5.1,0.1)
table = F_hat2d(n, x, test_stats)
print(table)

6. $\textbf{Computing the estimate of number of true null hypothese:}$ 

In [21]:
def num_of_null(p,m):
    """
    Input parameters:
    -------
    p: Sorted pvalues of the m null hypotheses
    m: Number of null hypothese that we are testing

    Output result:
    -------
    This function will give an estimate of m_0, the number of true null hypotheses.
    Knowing \widehat{m_0} enables us to carry out adaptive BH procedure. 

    """
    S = (1-p) / (m+1-np.arange(m))
    j = np.where(np.diff(S)<0)[-1][0]
    result = min(1/S[j] + 1, m)
    return result

In [22]:
# Example:

p = np.array([0.123,0.012,0.876,0.986,0.00123,0.098,0.98,0.99,0.567,0.876,0.212])
sorted_pvalues = np.sort(p)
m = len(p)
num_of_null(sorted_pvalues, m)

11

7. $\textbf{Computing the dfBB trajectory give a fbm trajectory:}$ 

In [33]:
def dfBB(traj,n):
    """
    Input parameters:
    -------
    traj: A 1D fractional Brownian trajectory
    n: Size of input trajectory

    Output result:
    -------
    This function will return the differenced fractional Brownian bridge trajectory of the input fBM trajectory.

    """
    first_element = traj[0]
    last_element = traj[-1]
    indices = np.arange(len(traj))
    fBB = traj - first_element - (indices / n) * (last_element - first_element)
    DFBB = np.diff(fBB)
    return DFBB

In [24]:
# Example: Compute the dfBB trajectory of a given fbm trajectory of size 20 with h = 0.125

sub_fbm = FractionalBrownianMotion(hurst = 0.125, t = 20)
sub_fbm_traj = sub_fbm.sample(20)
dfBB(sub_fbm_traj, 20)

array([-0.97388084, -0.19499523, -0.36884033, -0.17443622, -1.55404247,
        1.72596251, -0.06693747, -1.15564179,  1.66154667,  0.41607108,
       -0.55290371,  0.07748312, -0.51159915,  0.11624413,  0.95702439,
       -0.68897356, -0.41331186, -0.1718313 ,  0.78009875,  1.09296327])

8. $\textbf{Computing the sample autocorrelation give a dfBB trajectory:}$ 

In [34]:
def sample_autocorrelation(traj,n) : 
    """
    Input parameters:
    -------
    traj: A 1D diffrenced fractional Brownian bridge trajectory
    n: Size of input trajectory + 1

    Output result:
    -------
    This function will return the sample autocorrelation of the input dfBB trajectory. 

    """
    output_array = np.zeros(n) 
    for lag in range(0,n): 
        squared_distances = [traj[i] * traj[i-lag] for i in range(lag,n)]
        output_array[lag] = np.sum(squared_distances)
        autocorrelation = output_array / output_array[0]
    return autocorrelation


In [29]:
# Example: Compute the sample autocorrelation give a dfBB trajectory:
dfbb = np.array([-0.97388084, -0.19499523, -0.36884033, -0.17443622, -1.55404247,
        1.72596251, -0.06693747, -1.15564179,  1.66154667,  0.41607108,
       -0.55290371,  0.07748312, -0.51159915,  0.11624413,  0.95702439,
       -0.68897356, -0.41331186, -0.1718313 ,  0.78009875,  1.09296327])
sample_autocorrelation(dfbb, 20)


array([ 1.00000000e+00, -2.16037595e-01, -2.54627052e-01,  2.83703695e-01,
       -1.15697613e-01, -2.57692353e-02,  1.17866004e-01, -2.08972296e-01,
       -5.98510110e-02,  8.05942362e-02, -5.04129067e-04,  7.41677547e-02,
       -4.91021790e-02,  1.05844607e-01,  3.97557310e-03, -6.88861994e-02,
       -2.86191732e-03, -2.62063889e-02, -6.57249137e-02, -7.19113402e-02])

9. $\textbf{Objective function of hurst exponent estimate:}$ 

In [35]:
def objective_hurst_estimate(traj,h,n):
    """
    Input parameters:
    ------
    traj: A 1D fractional Brownian trajectory
    h: Hurst exponent of that trajectory
    n: Size of the input trajectory

    Output result:
    ------
    This is the objective function when we do the hurst exponent estimation. 
    The hurst exponent estimate \widehat{h} is the global minimum of this function on (0,1)

    """
    pho_hat = sample_autocorrelation(dfBB(traj,n),n)
    func = np.zeros(n-1)
    for i in range(n-1):
        func[i] = (pho_hat[i+1]-(0.5 * ((i+2)**(2*h)-2*((i+1)**(2*h))+i**(2*h)) + n**(2*h-2) + ((i+1)**(2*h)-(n-i-1)**(2*h)-n**(2*h))/(n*(n-i-1)))/(1-n**(2*h-2)))**2
    return np.sum(func)

In [ ]:
# No example running for this function, as we are only interested in the minimizer of the function, not the function itself. 

10. $\textbf{Hurst exponent estimation:}$ 

In [31]:
def hurst_exponent_estimate(traj,n):
    """
    Input parameter:
    ------
    traj: A set of 1D fractional Brownian trajectories.
    n: Size of input trajectories
    
    Output result:
    ------
    This function will return the hurst exponent estimate of the input trajectories. 

    """
    
    hurst = np.zeros(len(traj))
    for i in range(len(traj)):
        given_traj = traj[i,:]
        fixed_traj_function = lambda h: objective_hurst_estimate(given_traj,h,n)
        result = minimize_scalar(fixed_traj_function, bounds = (0,1), method = 'bounded')
        hurst[i] = result.x
    return hurst 

In [42]:
# Example: Let's estimate the hurst exponent of 10 fbm trajectories of size 1000 with h = 0.32

sub_fbm = FractionalBrownianMotion(hurst = 0.32, t = 1000)
sub_fbm_traj = np.stack([sub_fbm.sample(1000) for i in range(10)])
hurst_exponent_estimate(sub_fbm_traj, 1000)

array([0.28548465, 0.32931407, 0.31197575, 0.33077331, 0.30766873,
       0.31729826, 0.28165634, 0.33075118, 0.28162026, 0.31502437])

11. $\textbf{Table of hurst exponent thresholds for different n:}$ 

In [45]:
def hurst_threshold(size, num_of_MC = 100):
    """
    Input parameter:
    ------
    size: This is an array of consecutive integers, starts from 2, ends with 1+len(n). For example, size could be np.arange(2,101).
    num_of_MC: Number of Monte Carlo Simulations. By default, it is 100.


    Output result:
    ------
    This function will give the thresholds of using hurst exponent estimates to do classification of different size of trajectories.


    """
    result = np.zeros([len(size), 2])
    for i in range(len(size)):
        std_bm = FractionalBrownianMotion(hurst = 0.5, t = int(i+2))
        std_bm_traj = np.stack([std_bm.sample(int(i+2)) for k in range(num_of_MC)])
        hurst_exponent = hurst_exponent_estimate(std_bm_traj, i+2)
        sort_hurst_exponent = np.sort(hurst_exponent)
        result[i,:] = np.array([sort_hurst_exponent[2], sort_hurst_exponent[97]])
    return result

In [ ]:
# Example: 

n = np.arange(2,101)
hurst_threshold(n)
